In [31]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')
mne.set_log_level('warning')

In [32]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs

Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Left: 73Visual/Right: 71
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [33]:
epochs.apply_baseline((None, 0))

Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Left: 73Visual/Right: 71
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [34]:
# epochs.plot()

### Reject Artifacts based on channel signal amplitude

In [35]:
reject_criteria = dict(mag = 3000 * (10 ** - 15),
                      grad = 3000 * (10 ** - 13),
                      eeg = 150 * (10 ** - 6),
                      eog = 200 * (10 ** - 6))

flat_criteria = dict(mag = 1 * (10 ** - 15),
                    grad = 1 * (10 ** - 13),
                    eeg = 1 * (10 ** - 6))

In [36]:
epochs.drop_bad(reject = reject_criteria, flat = flat_criteria)

Number of events,271
Events,Auditory/Left: 57Auditory/Right: 61Button: 15Smiley: 14Visual/Left: 67Visual/Right: 57
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [37]:
epochs.plot_drop_log()

<Figure size 640x476 with 1 Axes>

In [38]:
epochs['Visual'].plot_image()

[<Figure size 640x476 with 3 Axes>,
 <Figure size 640x476 with 3 Axes>,
 <Figure size 640x476 with 3 Axes>]

In [39]:
epochs.plot_sensors(ch_type = 'eeg')

<Figure size 640x640 with 1 Axes>

In [40]:
epochs['Visual'].plot_image(picks = 'EEG 060')

[<Figure size 640x476 with 4 Axes>]

### SSP

In [41]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject = '01',
                             session = '01',
                             task = 'audiovisual',
                             run = '01',
                             datatype = 'meg',
                             root = bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq = 0.1, h_freq = 40)

ecg_projs, ecg_events = mne.preprocessing.compute_proj_ecg(raw, n_grad = 1, n_mag = 1, n_eeg = 0,
                                                           average = True)

eog_projs, eog_events = mne.preprocessing.compute_proj_eog(raw, n_grad = 1, n_mag = 1, n_eeg = 0,
                                                           average = True)

C:\Users\hp\AppData\Roaming\Python\Python37\site-packages\mne_bids\read.py:406: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw.set_channel_types(channel_type_dict)


In [42]:
ecg_projs

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | ECG-planar--0.200-0.400-PCA-01, active : False, n_channels : 203>,
 <Projection | ECG-axial--0.200-0.400-PCA-01, active : False, n_channels : 102>]

In [43]:
eog_projs

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 203>,
 <Projection | EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102>]

In [44]:
projs = ecg_projs + eog_projs
projs

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | ECG-planar--0.200-0.400-PCA-01, active : False, n_channels : 203>,
 <Projection | ECG-axial--0.200-0.400-PCA-01, active : False, n_channels : 102>,
 <Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 203>,
 <Projection | EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102>]

In [45]:
epochs.plot()

<MNEBrowseFigure size 1920x936 with 4 Axes>

In [47]:
epochs.add_proj(projs)
epochs.plot()

<MNEBrowseFigure size 1920x932 with 5 Axes>

In [46]:
epochs_cleaned = epochs.copy().apply_proj()

epochs_cleaned['Visual'].plot_image()
epochs_cleaned['Visual'].plot_image(picks = 'EEG 060')

[<Figure size 640x476 with 4 Axes>]

### ICA

In [49]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject = '01',
                             session = '01',
                             task = 'audiovisual',
                             run = '01',
                             datatype = 'meg',
                             root = bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq = 1, h_freq = 40)

C:\Users\hp\AppData\Roaming\Python\Python37\site-packages\mne_bids\read.py:406: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw.set_channel_types(channel_type_dict)


<Raw | sub-01_ses-01_task-audiovisual_run-01_meg.fif, 376 x 166800 (277.7 s), ~481.8 MB, data loaded>

In [51]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs_selection = epochs.selection
epochs_selection

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [54]:
events, event_id = mne.events_from_annotations(raw)
events.shape

(320, 3)

In [55]:
events = events[epochs_selection]
events.shape

(320, 3)

In [57]:
tmin = epochs.tmin
tmax = epochs.tmax
baseline = epochs.baseline

epochs_ica = mne.Epochs(raw,
                       events = events,
                       event_id = event_id,
                       tmin = tmin,
                       tmax = tmax,
                       baseline = baseline,
                       preload = True)

### Finally, fit ICA!

In [58]:
epochs_ica.info

<Info | 23 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 bads: 2 items (MEG 2443, EEG 053)
 ch_names: MEG 0113, MEG 0112, MEG 0111, MEG 0122, MEG 0123, MEG 0121, MEG ...
 chs: 204 GRAD, 102 MAG, 9 STIM, 60 EEG, 1 EOG
 custom_ref_applied: False
 description: acquisition (megacq) VectorView system at NMR-MGH
 dev_head_t: MEG device -> head transform
 dig: 146 items (3 Cardinal, 4 HPI, 61 EEG, 78 Extra)
 events: 1 item (list)
 experimenter: MEG
 file_id: 4 items (dict)
 highpass: 1.0 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 line_freq: 60
 lowpass: 40.0 Hz
 meas_date: 2002-12-03 19:01:10 UTC
 meas_id: 4 items (dict)
 nchan: 376
 proj_id: 1 item (ndarray)
 proj_name: test
 projs: PCA-v1: on, PCA-v2: on, PCA-v3: on
 sfreq: 600.6 Hz
 subject_info: 5 items (dict)
>

In [61]:
!pip install python-picard

In [63]:
n_components = 0.8
method = 'picard'
max_iter = 100
fit_params = dict(fastica_it = 5)
random_state = 42

ica = mne.preprocessing.ICA(n_components = n_components,
                           method = method,
                           max_iter = max_iter,
                           fit_params = fit_params,
                           random_state = random_state)

ica.fit(epochs_ica)

<ipython-input-63-239b3910ab08>:13: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs_ica)


<ICA | epochs decomposition, fit (picard): 153920 samples, 28 components, channels used: "mag"; "grad"; "eeg">

In [65]:
ica.plot_components(inst = epochs)

[<MNEFigure size 1920x966 with 20 Axes>, <MNEFigure size 975x492 with 8 Axes>]